In [8]:
from src.sapt5.fcOptimisGA.RealChromosome import Chromosome
def fcEval(communities, param):
    noNodes = param['noNodes']
    mat = param['mat']
    degrees = param['degrees']
    noEdges = param['noEdges']  
    M = 2 * noEdges
    Q = 0.0
    for i in range(0, noNodes):
        for j in range(0, noNodes):
            if (communities[i] == communities[j]):
               Q += (mat[i][j] - degrees[i] * degrees[j] / M)
    return Q * 1 / M

In [9]:
import networkx as nx
# read the network details
def readNet(fileName):
    f = open(fileName, "r")
    net = {}
    n = int(f.readline())
    net['noNodes'] = n
    mat = []
    for i in range(n):
        mat.append([])
        line = f.readline()
        elems = line.split(" ")
        for j in range(n):
            mat[-1].append(int(elems[j]))
    net['mat'] = mat 
    degrees = []
    noEdges = 0
    for i in range(n):
        d = 0
        for j in range(n):
            if mat[i][j] == 1:
                d += 1
            if (j > i):
                noEdges += mat[i][j]
        degrees.append(d)
    net["noEdges"] = noEdges
    net["degrees"] = degrees
    f.close()
    return net

def readGMLFile(fileName):
    f = open(fileName, "r")
    net = {}
    creator_name = f.readline();
    n = 0
    m = 0
    mat = []
    f.readline()  # graph
    f.readline()  # [
    f.readline()  # directed 0
    line = f.readline();
    while line != ']':
        line = line.replace("\n", "")
        line = line.strip()
        if line == 'node':
            # adding node
            f.readline()  # [
            id_line = f.readline()  # id number
            id = int(id_line.split()[1])
            label_line = f.readline()  # label string
            value_line = f.readline()  # label string
            f.readline()  # ]
            n = n + 1
        elif line == 'edge':
            if m == 0:
                for i in range(n):
                    mat.append([])
                    for j in range(n):
                        mat[i].append(0)
            # adding edge
            f.readline()  # [
            source_line = f.readline()  # source number
            source = int(source_line.split()[1])
            source = source - 1
            target_line = f.readline()  # target number
            #value_line = f.readline()  # label string
            target = int(target_line.split()[1])
            target = target - 1
            f.readline()  # ]
            mat[source][target] = 1
            mat[target][source] = 1
            m = m + 1
        line = f.readline()
        line = line.strip()
        line = line.replace("\n", "")
    degrees = []
    print(mat)
    for i in range(n):
        d = 0
        for j in range(n):
            if mat[i][j] == 1:
                d += 1
        degrees.append(d)
    net['noNodes'] = n
    net['noEdges'] = m
    net['degrees'] = degrees
    net['mat'] = mat
    return net


**GA running**

Use the GA (with real encoding) for identify the optimal solution


In [10]:



class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__problParam)
            self.__population.append(c)
    
    def evaluation(self,network):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres, network)
            
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best
        
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best

    def selection(self):
        pos1 = randint(0, self.__param['popSize'] - 1)
        pos2 = randint(0, self.__param['popSize'] - 1)
        if (self.__population[pos1].fitness > self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 
        
    
    def oneGeneration(self):
        newPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.binomial_crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        
    def oneGenerationSteadyState(self,network):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutationNew()
            off.fitness = self.__problParam['function'](off.repres, network)
            worst = self.worstChromosome()
            if off.fitness < worst.fitness:
                off = worst
            newPop.append(off)
        self.__population = newPop
        self.evaluation(network)

In [11]:
MIN=1
MAX=6
#from BinChromosome import Chromosome
from random import seed, randint 
import os
class Solver(object):
    def __init__(self):
        self._filename=""
        self._popSize=0
        self._noGen=0
    def setSolverParam(self,filename,popSize,noGen):
        self._filename=filename
        self._popSize=popSize
        self._noGen=noGen
    def solveGA(self):
        #generate seed
        seed()
        #read net
        crtDir =  os.getcwd()
        filePath = os.path.join(crtDir, 'realNetworks', self._filename)
        network = readGMLFile(filePath)
        # initialise de GA parameters
        gaParam = {'popSize' : self._popSize, 'noGen' :self._noGen, 'pc' : 0.8, 'pm' : 0.1}
        # problem parameters
        problParam = {'cr' : 0.5, 'min' : MIN, 'max' : MAX, 'function' : fcEval, 'noDim' : network['noNodes'], 'noBits' : 8,'alpha':0.2,'mat':network['mat']}
    
        # store the best/average solution of each iteration (for a final plot used to anlyse the GA's convergence)
        bestRepres = []
        bestFitness = -999999
    
        ga = GA(gaParam, problParam)
        ga.initialisation()
        ga.evaluation(network)
        
        for g in range(gaParam['noGen']):
            #plotting preparation
            bestSolX = ga.bestChromosome().repres
            bestSolY = ga.bestChromosome().fitness
            if bestSolY > bestFitness :
                bestFitness = bestSolY
                bestRepres.clear()
                for elem in bestSolX:
                    bestRepres.append(elem)
        
            #logic alg
            ga.oneGenerationSteadyState(network)
            #ga.oneGenerationElitism()
            # ga.oneGenerationSteadyState()
        
            bestChromo = ga.bestChromosome()
            print('Best solution in generation ' + str(g) + ' is: x = ' + str(bestChromo.repres) + ' f(x) = ' + str(bestChromo.fitness)) 
    
        print('\n\nBest solution is x = ' + str(bestRepres) + ' f(x) = ' + str(bestFitness))
        number_of_groups = 0
        groups = {}
        for el in bestRepres:
            if groups.get(el) is None :
                groups[el] = 1
                number_of_groups += 1
            
        print('The number of groups is : ' + str(number_of_groups) + ' and the associations to groups are (Node : Group):')
        for i in range(len(bestRepres)):
            print(str(i) + ' : ' + str(bestRepres[i]))
        plotInitial(network)
        plotDivision(network,bestRepres)
        

In [12]:
import numpy as np 
import matplotlib.pyplot as plt 
import warnings 
def plotInitial(network):
    warnings.simplefilter('ignore')

    A=np.matrix(network["mat"])
    G=nx.from_numpy_matrix(A)
    pos = nx.spring_layout(G)  # compute graph layout
    plt.figure(figsize=(10, 10))  # image is 8 x 8 inches 
    nx.draw_networkx_nodes(G, pos, node_size=200, cmap=plt.cm.RdYlBu)
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.show(G)


In [13]:
# plot a particular division in communities
def plotDivision(network,bestRepres):
    A=np.matrix(network["mat"])
    G=nx.from_numpy_matrix(A)
    pos = nx.spring_layout(G)  # compute graph layout
    plt.figure(figsize=(10, 10))  # image is 8 x 8 inches 
    nx.draw_networkx_nodes(G, pos, node_size = 200, cmap = plt.cm.RdYlBu, node_color = bestRepres)
    nx.draw_networkx_edges(G, pos, alpha = 0.3)
    plt.show(G)


In [14]:
from src.sapt5.UI import UI
solver=Solver()
UI=UI(solver)
UI.run()

[[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

KeyboardInterrupt: 